## 0.模型保存路径说明：

/root/autodl-tmp/models：其中保存的是 ollama 下载的模型

/root/autodl-tmp/Qwen3：其中保存的是从魔塔社区上下载的模型

/root/autodl-tmp/Qwen3/Qwen3-4B：是从魔塔社区上下载的官方的模型

/root/autodl-tmp/Qwen3-4B-GGUF：是从魔塔社区上下载的 LM Studio 发布的模型（试验效果很差，完全瞎回答，可能是配置文件的问题，就没有下载配置文件，哈哈）



## 1.Qwen3模型下载与环境搭建

conda create --name qwen3 python=3.12 -c https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/mai

conda init

source ~/.bashrc

conda activate qwen3

conda install jupyterlab

conda install ipykernel

python -m ipykernel install --user --name qwen3 --display-name "Python qwen3"

pip install modelscope

cd /root/autodl-tmp

mkdir Qwen3

cd /root/Qwen3

pip install -r requirements.txt

使用清华的镜像更快：pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple

cd /root/autodl-tmp/Qwen3

mkdir ./Qwen3-4B

modelscope download --model Qwen/Qwen3-4B --local_dir ./Qwen3-4B

## 2. Jupyter中使用transformer原生库调用流程

* 导入相关库

In [10]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

* 设置模型下载地址

In [11]:
model_name = "/root/autodl-tmp/Qwen3/Qwen3-4B"

* 实例化预训练模型与分词器

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
model_name,
torch_dtype="auto", # 指定模型权重数据类型
device_map="auto" # 指定模型权重加载设备
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.67it/s]


* 创建消息message

In [13]:
prompt = "给出一个关于大语言模型的简短介绍。"
messages = [
{"role": "user", "content": prompt}
]

* 词嵌入过程

In [14]:
text = tokenizer.apply_chat_template(
messages,
tokenize=False,
add_generation_prompt=True,
enable_thinking=True # 是否启用思考模式
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

* 创建并回复

In [15]:
# 进行文本补全
generated_ids = model.generate(
**model_inputs,
max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
# 结果将首先显示思考内容，在<think></think>标签中，然后是实际响应
print(tokenizer.decode(output_ids, skip_special_tokens=True))

<think>
好的，用户让我给出一个关于大语言模型的简短介绍。首先，我需要确定用户的需求是什么。他们可能是在做学术研究，或者需要向非专业人士解释这个概念，或者是在准备演讲稿。不管怎样，用户需要的是简洁明了的介绍，所以得抓住关键点。

首先，大语言模型（LLM）是什么？应该从定义开始，说明它们是基于大量文本数据训练的模型，能够理解和生成自然语言。接下来，可能需要提到它们的应用场景，比如文本生成、问答、编程等。然后，技术基础，比如深度学习和Transformer架构，这些是关键的技术点。

还要考虑用户可能的背景。如果他们不是技术背景，可能需要避免太多专业术语，或者用更通俗的语言解释。比如，Transformer架构可以简单说成是处理语言的先进模型结构，而不是深入讲解细节。

另外，用户可能想知道大语言模型的优势，比如强大的语言理解和生成能力，多任务处理能力，以及在各个领域的应用潜力。同时，可能需要提到它们的局限性，比如数据依赖性，或者可能的偏见问题，但用户要求的是简短介绍，所以可能不需要深入讨论缺点，除非用户特别提到。

还要检查是否有最新的发展，比如像GPT-4、BERT等模型，但可能不需要具体名字，除非用户要求。不过用户只是要简短介绍，所以可能不需要具体例子，但可以提到像GPT、BERT这样的代表模型。

结构上，应该先定义，再讲技术基础，然后应用场景，最后可能提到影响或未来展望。不过要保持简洁，所以可能需要合并这些部分。

需要确保语言流畅，没有冗余。比如，避免重复提到“自然语言处理”或者“深度学习”，但可以适当提到。同时，可能需要强调大语言模型在推动AI发展中的作用，比如在自然语言处理、机器学习等领域的应用。

最后，检查是否符合简短的要求，确保信息全面但不过于冗长。可能需要控制在三到四句话内，或者分点说明，但用户可能希望是连贯的段落。
</think>

大语言模型（Large Language Models, LLMs）是基于深度学习技术训练出的AI系统，通过海量文本数据学习语言规律，具备理解和生成自然语言的能力。它们通常基于Transformer架构，能够处理多任务（如文本生成、问答、翻译、编程等），在自然语言处理、机器学习等领域广泛应用。LLMs通过预训练和微调技术，可适应不同场景，推动AI在内容创作、智能助手、数据分析等领域的快速发展。


## Qwen3-4B模型接入Ollama与推理流程

Ollama 是在 Github 上的一个开源项目，其项目定位是：一个本地运行大模型的集成框架，目前主
要针对主流的 LLaMA 架构的开源大模型设计，通过将模型权重、配置文件和必要数据封装进由
Modelfile 定义的包中，从而实现大模型的下载、启动和本地运行的自动化部署及推理流程。此外，
Ollama 内置了一系列针对大模型运行和推理的优化策略，目前作为一个非常热门的大模型托管平台，基
本主流的大模型应用开发框架如 LangChain 、 AutoGen 、 Microsoft GraphRAG 及热门项目
AnythingLLM 、 OpenWebUI 等高度集成。

### 1.Ollama项目安装

Ollama 项目支持跨平台部署，目前已兼容Mac、Linux和Windows操作系统。特别地对 Mac 和
Windows 用户提供了非常直观的预览版，包括了内置的 GPU 加速功能、访问完整模型库的能力，以及对
OpenAI 的兼容性在内的 Ollama REST API ，对用户使用尤为友好。
但无论使用哪个操作系统， Ollama 项目的安装过程都设计得非常简单。根据后续的课程的研发需求
以及真实企业的应用需求，我们建议大家使用 Linux 系统进行实践。同时课程也将选择以 Linux 版本为
例进行详细介绍。对于其他操作系统版本的安装，大家可以通过如下链接，根据自己的实际情况进行安
装体验：https://github.com/ollama/ollama

* 【安装方案一】Ollama在线安装

安装命令：curl -fsSL https://ollama.com/install.sh | sh

* 【安装方案二】Ollama离线安装

更为一般的情况下，推荐使用 Ollama 离线部署。我们可以在 Ollama Github 主页查看目前
Ollama 支持的各操作系统安装包：https://github.com/ollama/ollama/releases

若是 Ubuntu 操作系统，选择其中 ollama-linux-amd64.tgz 下载和安装即可。

下载完成后，需要先上传至服务器：

然后使用如下命令进行解压缩

    mkdir ./ollama

    tar -zxvf ollama-linux-amd64.tgz -C ./ollama

解压缩后项目文件bin和lib

在bin中，可以找到ollama命令的可执行文件。

此时，我们可以使用如下方式使用ollama：

    cd ./bin

    ./ollama help

此处若显示没有可执行权限，可以使用如下命令为当前脚本添加可执行权限：

    chmod +x ollama

而为了使用命令方便，我们也可以将脚本文件写入环境变量中。我们可以在主目录（root）下找
到.bashrc文件：

然后在 .bashrc 文件结尾写入ollama/bin文件路径：

    export PATH=$PATH:/root/autodl-tmp/ollama/bin

保存并退出后，输入如下命令来使环境变量生效：

    source ~/.bashrc

然后在任意路径下输入如下命令，测试ollama环境变量是否生效

    ollama help

### 2. Qwen3-4B GGUF格式模型权重下载

#### 2.1 【方案一】在线下载模型权重

    ollama run qwen3:4B

#### 2.2 【方案二】离线下载模型权重

除了使用ollama run命令外，ollama也支持调用手动下载的自定义模型，但需要是GGUF格式。目
前Qwen3模型已有各种不同量化版本的GGUF模型在魔搭社区和huggingface中上线了：https://models
cope.cn/models/lmstudio-community/Qwen3-14B-GGUF

其中每个GGUF量化的权重都包含多个版本

其中ollama调用的就是Q4_K_M量化版本。这里我们还是考虑下载Q4_K_M量化版本的GGUF权重进
行调用，具体执行流程如下

* 下载GGUF格式权重

    cd ~/autodl-tmp

    mkdir Qwen3-14B-GGUF

    modelscope download --model lmstudio-community/Qwen3-4B-GGUF --local_dir ./Qwen3-4B-GGUF --include "Qwen3-4B-Q4_K_M.gguf"
    
    
    或

    # modelscope download --model lmstudio-community/Qwen3-4B-GGUF --local_dir ./Qwen3-4B-GGUF --exclude "*Q8*" --exclude "*Q3*" --exclude "*Q6*"
    # modelscope download --model lmstudio-community/Qwen3-4B-GGUF README.md --local_dir ./dir

* 注册模型

然后需要创建一个file文件，命名为 ModelFile ，用于进行ollama模型注册

然后在File文件中写入自定义模型GGUF权重地址

    FROM /root/autodl-tmp/Qwen3-4B-GGUF/Qwen3-4B-Q4_K_M.gguf

然后，将该模型加入Ollama本地模型列表

    cd /root/Qwen3

    ollama create Qwen3-4B -f ModelFile

查看模型是否注册成功

    ollama list

接下来即可在命令行中进行调用

    ollama run Qwen3-4B-GGUF
    这个模型被量化的非常糟糕（效果）

---

# <center>Qwen-3本地部署与调用详解（下）

### 3. ollama API本地调用Qwen3

&emsp;&emsp;在部署完ollama之后，即可借助ollama API（也就是OpenAI风格API）在代码环境中调用模型。

- 导入OpenAI库及实例化OpenAI客户端

In [13]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',  # 必须传递该参数
)

prompt = "在单词\"strawberry\"中，总共有几个R？"
messages = [
    {"role": "user", "content": prompt}
]

- 创建消息并获得回复

In [15]:
response = client.chat.completions.create(
    messages=messages,
    model='qwen3:4B',
)

print(response.choices[0].message.content)

在英文单词“strawberry”中，字母“r”（小写）的总出现次数是3次。以下是详细分解：

- 单词“strawberry”的拼写：s - t - r - a - w - b - e - r - r - y
- 字母“r”出现在以下位置（索引从1开始）：
  - 第3个字母：r
  - 第8个字母：r
  - 第9个字母：r

因此，**总共有3个r**。

注意：在字母计数中，通常不区分大小写（本单词均为小写），问题中“R”指代字母“r”。标准英语拼写下，此计数是准确的。  
（常见误解：有些人可能误认为“berry”部分只有1个r，但实际上“berry”拼写为b-e-r-r-y，包含2个r。）

**答案：3**


&emsp;&emsp;因为Qwen3刚刚发布，所以Ollama目前还不支持在API调用时通过`enable_thinking`参数禁用思考过程，这里有一种策略是先用`/no_think` 提示不要进入思考过程。如下所示：

In [16]:
prompt = "/no_think 在单词\"strawberry\"中，总共有几个R？"
messages = [
    {"role": "user", "content": prompt}
]


response = client.chat.completions.create(
    messages=messages,
    model='qwen3:4B',
)

print(response.choices[0].message.content)

在单词 "strawberry" 中，字母 "R" 的出现次数（不区分大小写，但单词本身是小写）如下：

- 拼写：s-t-r-a-w-b-e-r-r-y
- 位置：第 3 个字母是 "r"，第 8 个字母是 "r"，第 9 个字母是 "r"。

因此，总共有 **3 个 R**。

**注意**：问题中的 "/no_think" 可能是用户自定义的前缀或测试标签（意为“不要思考”），但不影响单词的拼写计数。基于标准英语拼写，答案已确认。

最终答案：**3**


&emsp;&emsp;而此时显存占用约22G左右（14B Q4量化模型）：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291258998.png" width=60%></div>


## 四、Qwen3模型接入vLLM与推理流程

&emsp;&emsp;接下来继续介绍Qwen3-14B模型借助vLLM进行推理的完整流程。相比ollama，vLLM更加适合企业级高并发应用场景，但对应的，显存占用也会更高，vLLM项目主页：https://github.com/vllm-project/vllm`Vllm` 底层是基于`Pytorch` 构建，其`Gtihub` 开源地址为：https://github.com/vllm-project/vllm 

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181137533.png" width=60%></div>

&emsp;&emsp;从各种基准测试数据来看，同等配置下，使用 `vLLM` 框架与 `Transformer` 等传统推理库相比，其吞吐量可以提高一个数量级，这归功于以下几个特性：

- **高级 GPU 优化**：利用 `CUDA` 和 `PyTorch` 最大限度地提高 `GPU` 利用率，从而实现更快的推理速度。`Ollama`其实是对`CPU-GPU`的混合应用，但`vllm`是针对纯`GPU`的优化。
- **高级内存管理**：通过`PagedAttention`算法实现对 `KV cache`的高效管理，减少内存浪费，从而优化大模型的运行效率。
- **批处理功能**：支持连续批处理和异步处理，从而提高多个并发请求的吞吐量。
- **安全特性**：内置 `API` 密钥支持和适当的请求验证，不像其他完全跳过身份验证的框架。
- **易用性**：`vLLM` 与 `HuggingFace` 模型无缝集成，支持多种流行的大型语言模型，并兼容 `OpenAI` 的 `API` 服务器。

### 4.1 vLLM安装与启动

&emsp;&emsp;使用`vllm`框架部署`Qwen3`模型，同样需要先安装`Python`运行环境，这里我们可以复用之前安装的`Anaconda`环境，具体执行命令如下：

```bash
    conda activate qwen3
```

&emsp;&emsp;注意：`vllm`官方要求的是`Python 3.9` ~ `Python 3.12` 之间的版本，所以如果`Anaconda`环境版本不在此范围内，请务必重新创建符合要求的`Anaconda`环境。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504271905428.png" width=60%></div>


&emsp;&emsp;然后，安装`vLLM`框架，具体执行命令如下：

```bash
    pip install vllm -i https://pypi.tuna.tsinghua.edu.cn/simple
```


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504271905429.png" width=60%></div>

&emsp;&emsp;此时耐心等待安装完成即可。待安装完成后，可以使用`pip show vllm` 命令查看`vllm` 框架的安装信息，可以明确看到当前安装的`vllm` 版本号。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504271909847.png" width=60%></div>

&emsp;&emsp;目前vLLM已支持Qwen3模型调用，可以在模型支持列表中查看模型关键字：https://docs.vllm.ai/en/latest/models/supported_models.html

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291124292.png" width=60%></div>

&emsp;&emsp;接下来即可按照如下流程进行调用。

### 4.2 OpenAI风格API响应模式


&emsp;&emsp;需要注意的是，随着模型上下文越长，所需要占用的显存也越大。根据测试，14B模型在32K上下文时，运行需要约30G显存。实际运行命令类似如下形式：

```bash
    vllm serve Qwen3-8B --max-model-len 32768 --host 192.168.110.131 --port 8000
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291320578.png" width=60%></div>


&emsp;&emsp;启动vLLM时候需要谨慎的设置最大上下文和对应的运行GPU数量。

- 单GPU运行命令

```bash
    cd /root/autodl-tmp
    vllm serve ./qwen3-8B --max-model-len 32768
```

- 双GPU运行命令

```bash
    cd /root/autodl-tmp
    CUDA_VISIBLE_DEVICES=0,1 vllm serve ./qwen3-8B --tensor-parallel-size 2
```

&emsp;&emsp;启动成功后，即可在Jupyter中进行调用。

- 导入OpenAI库并实例化OpenAI客户端

In [18]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://192.168.110.131:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

- 创建消息获得回复

In [19]:
prompt = "在单词\"strawberry\"中，总共有几个R？"
messages = [
    {"role": "user", "content": prompt}
]

In [20]:
response = client.chat.completions.create(
    model="Qwen3:8B",
    messages=messages,
)

print(response.choices[0].message.content)

<think>
嗯，用户问的是在单词“strawberry”中有几个字母R。首先，我需要确认单词的正确拼写，可能有时候会有拼写错误或者变体，但“strawberry”应该是正确的拼写。

接下来，我应该把单词分解开来，逐个字母检查。先写下单词：S-T-R-A-W-B-E-R-R-Y。或者可能我记错了，应该再仔细拼一遍。S-T-R-A-W-B-E-R-R-Y？或者是不是中间有其他字母？比如，strawberry的正确拼写是S-T-R-A-W-B-E-R-R-Y，对吗？是的，没错，正确的拼写是S-T-R-A-W-B-E-R-R-Y，也就是“strawberry”。

现在，我需要数一下里面有几个R。先从头开始，每个字母依次检查：

S - 不是R
T - 不是
R - 是第一个R
A - 不是
W - 不是
B - 不是
E - 不是
R - 第二个R
R - 第三个R
Y - 不是

所以看起来有三个R？但可能我哪里数错了。或者是不是中间的拼写有误？

或者可能用户的问题有陷阱？比如，是否在拼写中有错误？比如，正确的拼写是strawberry，其中R出现的位置是第三个字母，然后是第8个和第9个字母？或者是不是中间的拼写有误？

让我再仔细拼一遍：S-T-R-A-W-B-E-R-R-Y。对的，S是第一个字母，然后T，R，A，W，B，E，R，R，Y。所以R出现在第3、8、9的位置，也就是三个R？

或者是不是我数错了？比如，第三个字母是R，然后在第八个和第九个字母也是R？所以总共有三个R？

但可能用户可能有不同的答案，或者我哪里出错了？

或者是不是在单词中R出现两次？比如，strawberry的正确拼写是S-T-R-A-W-B-E-R-R-Y，对吗？所以第三个字母是R，然后第八和第九是R，所以三个R？

或者是不是在拼写中R只出现两次？比如，可能用户认为是strawberry中R出现两次？比如，可能我记错了拼写？

或者是不是应该拆分单词为straw + berry？straw是S-T-R-A-W，berry是B-E-R-R-Y，所以整个单词是strawberry，所以R在straw中的第三个字母，然后在berry中的第三个和第四个字母？所以总共有三个R？

是的，这样的话，strawberry中的R出现三次。所以答案应该是三个R？

不过，可能用户会误以为是两次，所

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/5429b86d22f29729ef960ff48e115fe.jpg" alt="5429b86d22f29729ef960ff48e115fe" style="zoom:50%;" />

## 五、基于llama.cpp的QwQ模型CPU推理

&emsp;&emsp;Qwen3系列模型目前也是可以使用llama.cpp进行纯CPU推理或者CPU+GPU混合推理的。接下来介绍如何使用llama.cpp调用模型权重进行推理和对话。

### 1. llama.cpp下载与编译

- llama.cpp项目主页：https://github.com/ggml-org/llama.cpp

<div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250221174445079.png" width=60%></div>

&emsp;&emsp;由于llama.cpp是个C语言项目，因此实际调用过程需要先构建项目，然后设置参数进行编译，然后最终创建可执行文件（类似于脚本），再运行本地大模型。借助llama.cpp可以实现纯CPU推理、纯GPU推理和CPU+GPU混合推理。

- 依赖下载

  ```bash
  apt-get update
  apt-get install build-essential cmake curl libcurl4-openssl-dev -y
  ```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504281026269.png" width=60%></div>

&emsp;&emsp;这条命令安装了一些常用的构建和开发工具，具体的每个部分的含义如下：

  - **`build-essential`**：安装一组构建必需的工具和库，包括：
    - 编译器（如 GCC）
    - `make` 工具
    - 其他一些常见的构建工具，确保你的系统能进行编译。
  - **`cmake`**：安装 **CMake** 工具，它是一个跨平台的构建系统，允许你管理项目的编译过程。
  - **`curl`**：安装 **cURL** 工具，它是一个命令行工具，用于通过 URL 发送和接收数据。它在很多开发场景中都很有用，尤其是与网络交互时。
  - **`libcurl4-openssl-dev`**：安装 **libcurl** 库的开发版本。它是 cURL 的一个库文件，允许你在编程中通过 cURL 发送 HTTP 请求。`libcurl4-openssl-dev` 是与 **OpenSSL** 配合使用的版本，提供了 SSL/TLS 加密支持，用于安全的 HTTP 请求。

- llama.cpp源码下载

&emsp;&emsp;若是AutoDL服务器，可以先开启学术加速：

```bash
    source /etc/network_turbo
```

&emsp;&emsp;如果是其他服务器或者本地服务器，则可以直接进行源码下载：

 ```bash
  git clone https://github.com/ggerganov/llama.cpp
```

&emsp;&emsp;也可以直接在课件网盘中找到代码文件，直接上传服务器并解压缩：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250429200007375.png" alt="image-20250429200007375" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/5429b86d22f29729ef960ff48e115fe.jpg" alt="5429b86d22f29729ef960ff48e115fe" style="zoom:50%;" />

&emsp;&emsp;准备好后，即可在服务器中看到llama.cpp项目文件夹：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504281026270.png" width=60%></div>

&emsp;&emsp;接下来，即可开始进行项目构建与编译。

- 项目构建与编译

  ```bash
  cmake llama.cpp -B llama.cpp/build \
      -DBUILD_SHARED_LIBS=OFF -DGGML_CUDA=ON -DLLAMA_CURL=ON
  ```

- **`cmake`**：运行 CMake 工具，用于配置和生成构建文件。

- **`llama.cpp`**：指定项目的源代码所在的目录。在这个例子中，`llama.cpp` 是项目的根目录。

- **`-B llama.cpp/build`**：指定生成构建文件的目录。`-B` 参数表示**构建目录**，`llama.cpp/build` 是生成的构建目录。这是 CMake 将生成的文件存放的地方（例如 Makefile 或 Ninja 构建文件）。
- 同时还指定了一些编译选项：
- **禁用共享库**（`-DBUILD_SHARED_LIBS=OFF`），生成 **静态库**。
- **启用 CUDA 支持**（`-DGGML_CUDA=ON`），以便在有 GPU 的情况下使用 GPU 加速。
- **启用 CURL 库支持**（`-DLLAMA_CURL=ON`），以便支持网络请求。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504281026271.png" width=60%></div>

&emsp;&emsp;然后需要进一步进行编译：

  ```bash
    cmake --build llama.cpp/build --config Release -j --clean-first --target llama-quantize llama-cli llama-gguf-split
  ```


  - **`--build llama.cpp/build`**：告诉 CMake 使用 `llama.cpp/build` 目录中的构建文件来执行构建过程。这个目录是在之前运行 `cmake llama.cpp -B llama.cpp/build` 命令时生成的，包含了所有构建所需的文件（例如 Makefile 或 Ninja 构建文件）。
  - **`--config Release`**：指定构建的配置为 **Release** 配置。
    - **Release** 配置通常意味着启用更多的 **优化**，生成的程序运行速度较快，适合发布。
    - 在 CMake 中，通常有两种常见的构建配置：
      - **Debug**：用于调试版本，包含调试信息且没有做过多优化。
      - **Release**：优化后的发布版本，去除调试信息，运行时性能更高。
  - **`-j`**：表示并行构建，允许 CMake 使用多个 CPU 核心来加速构建过程。
    - 如果没有指定数字，CMake 会使用默认的并行级别，通常是可用的所有 CPU 核心。你也可以指定并行的作业数，例如 `-j 8` 表示使用 8 个并行作业进行编译。
  - **`--clean-first`**：表示在构建之前先清理掉之前的构建结果。这可以确保每次构建时都是从一个干净的状态开始，避免由于缓存或中间文件引起的编译错误。
    - 如果你之前运行过构建并且有问题，或者希望重新构建而不使用任何缓存文件，这个选项非常有用。
  - **`--target`**：指定构建的目标（target）。通常，一个项目会定义多个目标（比如库、可执行文件等），通过这个参数可以告诉 CMake 只编译特定的目标。
    - **`llama-quantize`**：可能是与模型量化相关的目标。量化（quantization）是将模型的精度从浮点数降低到整数，从而减少内存占用和提高推理速度。
    - **`llama-cli`**：可能是一个命令行工具，用于运行模型或与用户交互。
    - **`llama-gguf-split`**：可能是一个用于拆分模型文件的目标，通常用于将一个大模型文件拆分成多个小文件，方便存储和加载。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504281026272.png" width=60%></div>

- 复制可执行文件

  ```bash
  cp llama.cpp/build/bin/llama-* llama.cpp
  ```

  将 **所有生成的可执行文件** 从构建目录 `llama.cpp/build/bin/` 复制到项目的根目录 `llama.cpp` 下。这样可以更方便地在项目根目录下执行这些可执行文件，而无需每次都进入构建目录。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504281028231.png" width=60%></div>


&emsp;&emsp;在准备完成后，接下来即可进行调用和推理测试了。

### 2.借助llama.cpp运行Qwen3模型

- 纯CPU推理流程【1token/s】

&emsp;&emsp;首先是纯CPU推理测试。此时系统只调用内存+CPU进行计算，此时不会用到GPU。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291341031.png" width=60%></div>

&emsp;&emsp;此时模型运行门槛很低，但运行速度非常慢，只有不到1tokens/s。具体推理代码实现流程如下：

```bash
    cd ./llama.cpp

    ./llama-cli --model /home/08_vllm/Qwen3-14B-GGUF/Qwen3-14B-Q4_K_M.gguf --cache-type-k q4_0 --threads 64 --prio 2 --temp 0.6 --ctx-size 512 --seed 3407 --n-gpu-layers 0 -no-cnv --prompt "<｜User｜>你好，好久不见，请介绍下你自己。<｜Assistant｜>"
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291340108.png" width=60%></div>

&emsp;&emsp;其中命令行核心参数说明：

- `--threads`：CPU 核心数;
- `--ctx-size`：输出的上下文长度；
- `--n-gpu-layers` ：需要卸载到 GPU 的层数，设置为0时代表完全使用CPU进行推理；
- `--temp`：模型温度参数；
- `-no-cnv`：不进行多轮对话；
- `--cache-type-k`：K 缓存量化为 4bit；
- `--seed`：随机数种子；

&emsp;&emsp;实际运行效果如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291341632.png" width=60%></div>

- CPU+GPU混合推理

&emsp;&emsp;接下来进一步尝试CPU+GPU混合推理，我们只需要合理的设置`--n-gpu-layers`参数，即可灵活的将模型的部分层加载到GPU上进行运行。并且无需手动设置，llama.cpp会自动识别当前GPU数量以及可以分配的显存，自动将模型权重加载到各个不同的GPU上。例如，我们这里考虑将30层加载到GPU上，运行效果如下所示：

```bash
./llama-cli \
    --model /home/08_vllm/Qwen3-14B-GGUF/Qwen3-14B-Q4_K_M.gguf \
    --cache-type-k q4_0 \
    --threads 64 \
    --prio 2 \
    --temp 0.6 \
    --ctx-size 512 \
    --seed 3407 \
    --n-gpu-layers 30 \
    -no-cnv \
    --prompt "<｜User｜>你好，好久不见，请介绍下你自己。<｜Assistant｜>" 
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291344892.png" width=60%></div>

&emsp;&emsp;此时显存占用不到10G：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291344893.png" width=60%></div>


&emsp;&emsp;推理速度略微有所提升，能达到接近2tokens/s

- 纯GPU推理

&emsp;&emsp;最后，我们更进一步，尝试把全部的模型权重都放在GPU上进行推理。

```bash
./llama-cli \
    --model /home/08_vllm/Qwen3-14B-GGUF/Qwen3-14B-Q4_K_M.gguf \
    --cache-type-k q4_0 \
    --threads 64 \
    --prio 2 \
    --temp 0.6 \
    --ctx-size 512 \
    --seed 3407 \
    --n-gpu-layers 64 \
    -no-cnv \
    --prompt "<｜User｜>你好，好久不见，请介绍下你自己。<｜Assistant｜>" 
```

&emsp;&emsp;此时GPU占用约11G：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291347015.png" width=60%></div>

&emsp;&emsp;推理速度则能达到14tokens/s。


### 六、Qwen3 接入OpenWeb-UI

### 1.Open-WebUI部署流程

&emsp;&emsp;首先需要安装Open-WebUI，官网地址如下：https://github.com/open-webui/open-webui。

<div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214142632738.png" width=60%></div>

&emsp;&emsp;我们可以直接使用pip命令快速完成安装：


```bash
    pip install open-webui
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504281806967.png" width=60%></div>


&emsp;&emsp;可以直接使用在GitHub项目主页上直接下载完整代码包，并上传至服务器解压缩运行：



<div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214142010929.png" width=60%></div>

&emsp;&emsp;此外，也可以在课件网盘中领取完整代码包，并上传至服务器解压缩运行：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250429200151840.png" alt="image-20250429200151840" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/5429b86d22f29729ef960ff48e115fe.jpg" alt="5429b86d22f29729ef960ff48e115fe" style="zoom:50%;" />

&emsp;&emsp;在确保ollama正常运行的情况下，进行后续操作。

### 2. Open-WebUI启动与对话流程

&emsp;&emsp;在准备好了Open-WebUI和一系列模型权重后，接下来我们尝试启动Open-WebUI，并借助本地模型进行问答。

&emsp;&emsp;首先需要设置离线环境，避免Open-WebUI启动时自动进行模型下载：

```bash
    export HF_HUB_OFFLINE=1
```

&emsp;&emsp;然后启动Open-WebUI

```bash
    open-webui serve
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504281812378.png" width=60%></div>


&emsp;&emsp;需要注意的是，如果启动的时候仍然报错显示无法下载模型，是Open-WebUI试图从huggingface上下载embedding模型，之后我们会手动将其切换为本地运行的Embedding模型。


<div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214154657014.png" width=60%></div>


&emsp;&emsp;然后在本地浏览器输入地址:8080端口即可访问：


<div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214154949305.png" width=60%></div>


> 若使用AutoDL，则需要使用SSH隧道工具进行地址代理
>
><div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214155045849.png" width=60%></div> 


> 更多AutoDL相关操作详见公开课：《AutoDL快速入门与GPU租赁使用指南》|https://www.bilibili.com/video/BV1bxB7YYEST/
>
> <div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250205182609797.png" width=60%></div> 


&emsp;&emsp;然后首次使用前，需要创建管理员账号：

<div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214155158043.png" width=60%></div> 

&emsp;&emsp;然后点击登录即可。需要注意的是，此时Open-WebUI会自动检测后台是否启动了ollama服务，并列举当前可用的模型。稍等片刻，即可进入到如下页面：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291551362.png" width=60%></div>




&emsp;&emsp;接下来即可进入到对话页面：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291551363.png" width=60%></div> 

&emsp;&emsp;对话效果如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291551364.png" width=60%></div> 

### 3. 本地知识库检索

<div align=center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214171919602.png" width=60%></div>

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291551365.png" width=60%></div>

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291551366.png" width=60%></div>

### 4. 代码解释器

<div align=center>
    <img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291609658.png" width=60%>
</div>

<div align=center>
    <img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291609659.png" width=60%>
</div>

## 5. 调用外部工具

&emsp;&emsp;Qwen3全系列支持Function calling，因此我们可以基于此完成Open-WebUI的外部工具调用工作。

- Open-WebUI工具调用功能实现

<div align=center>
    <img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214174019551.png" width=60%>
</div>

&emsp;&emsp;然后编写天气查询函数：

```python
    import requests
    import json
    from fastapi import Request
    from open_webui.models.users import Users

    class Tools:
        def __init__(self):
            pass

        async def get_weather(
            self, loc: str, __request__: Request, __user__: dict, __event_emitter__=None
        ) -> str:
            """
            获取指定城市的即时天气信息。

            :param loc: 城市名称（如果是非英文城市名称，请先将其翻译为英文城市名称再输入）。
            :param __request__: HTTP 请求对象（来自 FastAPI）。
            :param __user__: 用户信息（可以用于个性化或记录请求）。
            :param __event_emitter__: 事件发射器，用于将实时状态更新发送到前端。
            :return: 格式化后的天气数据，作为字符串返回。
            
            说明：
            如果输入的是非英文城市名称，请先将其翻译为对应的英文城市名称再输入。
            比如，输入“中国，北京”，需要转为 "Beijing"。
            """

            # Step 1. 通知用户正在获取天气数据
            await __event_emitter__(
                {
                    "type": "status",
                    "data": {"description": f"正在获取 {loc} 的天气数据...", "done": False},
                }
            )

            try:
                # Step 2. 构建请求的 URL 和查询参数
                url = "https://api.openweathermap.org/data/2.5/weather"
                params = {
                    "q": loc,
                    "appid": "YOUR_API_KEY",  # 你的 API 密钥
                    "units": "metric",  # 使用摄氏度
                    "lang": "zh_cn",  # 输出简体中文
                }

                # Step 3. 向 OpenWeather API 发送 GET 请求
                response = requests.get(url, params=params)

                # Step 4. 解析响应数据
                data = response.json()

                # Step 5. 提取并格式化天气描述
                weather_data = data.get("weather", [])
                if weather_data:
                    main_weather = weather_data[0].get("main", "")
                    description = weather_data[0].get("description", "")
                    weather_info = f"当前天气：{main_weather} - {description}"
                else:
                    weather_info = "天气信息不可用。"

                # Step 6. 通知用户天气数据已经成功获取
                await __event_emitter__(
                    {
                        "type": "status",
                        "data": {"description": f"成功获取 {loc} 的天气数据", "done": True},
                    }
                )

                # Step 7. 返回格式化后的天气信息并发送到前端
                await __event_emitter__(
                    {
                        "type": "message",
                        "data": {"content": f"{loc} 的天气信息: {weather_info}"},
                    }
                )

                # 返回格式化后的天气数据作为字符串
                return f"{loc} 的天气信息: {weather_info}"

            except Exception as e:
                # Step 8. 如果发生错误，通知用户
                await __event_emitter__(
                    {
                        "type": "status",
                        "data": {"description": f"发生错误: {str(e)}", "done": True},
                    }
                )

                # 返回错误信息
                return f"获取天气数据时发生错误：{str(e)}"
```

&emsp;&emsp;并将函数进行写入：

<div align=center>
    <img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214174216021.png" width=60%>
</div>

&emsp;&emsp;完成后即可看到新的工具：

<div align=center>
    <img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250214174233430.png" width=60%>
</div>

&emsp;&emsp;在对话时可以开启天气查询函数：

<div align=center>
    <img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291609660.png" width=60%>
</div>

<div align=center>
    <img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504291609661.png" width=60%>
</div>

&emsp;&emsp;至此，我们就详细介绍了QwQ模型的各类本地部署与调用方法。